## NextGrowth Labs Assignment

Deployment of model for finding good reviews with 1 star rating. 

App Name: Google Chrome

App store: Google Play Store

App ID: com.android.chrome

### Importing relevant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

sns.set()

In [1]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import streamlit as st
from PIL import Image


### Creating requirements using pipreqs

In [16]:
!pipreqs --encoding utf-8 "./"

INFO: Successfully saved requirements file in ./requirements.txt


### Importing dataset for EDA

In [2]:
raw_data = pd.read_csv('chrome_reviews.csv')

In [3]:
raw_data.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [4]:
raw_data.describe(include = 'all')

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
count,7204.000000,7204,7203,7204.000000,7204.000000,7204,95,7119,7204,7204
unique,NaN,7204,4795,NaN,NaN,7099,89,111,68,1
top,NaN,https://play.google.com/store/apps/details?id=...,Good,NaN,NaN,A Google user,"Hey there, thanks for reaching out! We are awa...",87.0.4280.101,2020-12-24,com.android.chrome
freq,NaN,1,714,NaN,NaN,42,4,1088,280,7204
mean,252654.665047,NaN,NaN,3.592726,1.216685,NaN,NaN,NaN,NaN,NaN
std,204647.596623,NaN,NaN,1.728362,41.382835,NaN,NaN,NaN,NaN,NaN
min,3886.000000,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,83254.750000,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN
50%,181866.500000,NaN,NaN,5.000000,0.000000,NaN,NaN,NaN,NaN,NaN
75%,419485.250000,NaN,NaN,5.000000,0.000000,NaN,NaN,NaN,NaN,NaN


In [5]:
raw_data.isnull().sum()

ID                    0
Review URL            0
Text                  1
Star                  0
Thumbs Up             0
User Name             0
Developer Reply    7109
Version              85
Review Date           0
App ID                0
dtype: int64

In [6]:
raw_data.dropna(subset = ['Text'], inplace = True)

In [7]:
raw_data.isnull().sum()

ID                    0
Review URL            0
Text                  0
Star                  0
Thumbs Up             0
User Name             0
Developer Reply    7108
Version              85
Review Date           0
App ID                0
dtype: int64

In [8]:
raw_data

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome
...,...,...,...,...,...,...,...,...,...,...
7199,684987,https://play.google.com/store/apps/details?id=...,Bagusss..,5,0,boima panjaitan,NaN,88.0.4324.93,2021-03-05,com.android.chrome
7200,684988,https://play.google.com/store/apps/details?id=...,Bad version 😔,1,0,निशान्त सिंह,NaN,88.0.4324.181,2021-03-05,com.android.chrome
7201,684989,https://play.google.com/store/apps/details?id=...,One thing that I have to say I can't spelled t...,5,0,Virgie Allen,NaN,89.0.4389.72,2021-03-05,com.android.chrome
7202,684990,https://play.google.com/store/apps/details?id=...,Excellent,5,0,Kazuo Guevarra,NaN,89.0.4389.72,2021-03-05,com.android.chrome


In [9]:
raw_data.columns

Index(['ID', 'Review URL', 'Text', 'Star', 'Thumbs Up', 'User Name',
       'Developer Reply', 'Version', 'Review Date', 'App ID'],
      dtype='object')

In [10]:
#Creating Dataframe containing 1 stars from the raw_data
neg_df = raw_data.loc[raw_data['Star'] == 1]
neg_df = neg_df[['Review URL','Text','Star']]
neg_df.reset_index(drop = True,inplace = True)
neg_df

,Review URL,Text,Star
0,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1
1,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1
2,https://play.google.com/store/apps/details?id=...,Very bad app 😞,1
3,https://play.google.com/store/apps/details?id=...,Many times I tried to update its not updating....,1
4,https://play.google.com/store/apps/details?id=...,App is not getting update and it is not gettin...,1
...,...,...,...
1889,https://play.google.com/store/apps/details?id=...,It will not let me update Samsung Galaxy a20,1
1890,https://play.google.com/store/apps/details?id=...,"Every time updates screw things up, I mean thi...",1
1891,https://play.google.com/store/apps/details?id=...,"for every search google translator pops up, ev...",1
1892,https://play.google.com/store/apps/details?id=...,Bad version 😔,1


In [11]:
# Using regex to clean the data

import re

remove_caps = re.compile(r'[\W]')
remove_special = re.compile(r'[^a-z0-9\s]')

def normalizedtexts(text):
    cleanedtext = []
    for i in range(len(text)):
        lower = text[i].lower()
        no_caps = remove_caps.sub(r' ',lower)
        no_special = remove_special.sub(r' ',no_caps)
        cleanedtext.append(no_special) 
    return cleanedtext

In [12]:
neg_df['Cleaned_Text'] = normalizedtexts(neg_df['Text'])
neg_df

,Review URL,Text,Star,Cleaned_Text
0,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,not able to update neither able to uninstall
1,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,many unwanted ads
2,https://play.google.com/store/apps/details?id=...,Very bad app 😞,1,very bad app
3,https://play.google.com/store/apps/details?id=...,Many times I tried to update its not updating....,1,many times i tried to update its not updating ...
4,https://play.google.com/store/apps/details?id=...,App is not getting update and it is not gettin...,1,app is not getting update and it is not gettin...
...,...,...,...,...
1889,https://play.google.com/store/apps/details?id=...,It will not let me update Samsung Galaxy a20,1,it will not let me update samsung galaxy a20
1890,https://play.google.com/store/apps/details?id=...,"Every time updates screw things up, I mean thi...",1,every time updates screw things up i mean thi...
1891,https://play.google.com/store/apps/details?id=...,"for every search google translator pops up, ev...",1,for every search google translator pops up ev...
1892,https://play.google.com/store/apps/details?id=...,Bad version 😔,1,bad version



### Using Vader for Sentiment Analysis

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [14]:
#Adding the positive sentiment to the df
compound = []
neg = []
pos = []
neu = []
for i in range(len(neg_df)):
    sentiment = sia.polarity_scores(neg_df['Cleaned_Text'][i])
    
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    pos.append(sentiment['pos'])

    if sentiment['compound'] > 0.4:
        compound.append(sentiment['compound'])
       
    else:
        compound.append(-1)

neg_df['Compound Value Vader'] = compound
neg_df['Negative'] = neg
neg_df['Neutral'] = neu
neg_df['Positive'] = pos


In [15]:
final = neg_df.loc[(neg_df['Neutral'] > 0.0) & 
                   (neg_df['Positive'] > neg_df['Neutral']) & 
                   (neg_df['Negative'] == 0) & 
                   (neg_df['Compound Value Vader'] > 0.4)]
final.reset_index(drop = True,inplace = True)
#final.sort_values(['Compound Value Vader'], ascending = False)
final

,Review URL,Text,Star,Cleaned_Text,Compound Value Vader,Negative,Neutral,Positive
0,https://play.google.com/store/apps/details?id=...,Won't updated,1,won t updated,0.5719,0.0,0.351,0.649
1,https://play.google.com/store/apps/details?id=...,Good serving,1,good serving,0.4404,0.0,0.256,0.744
2,https://play.google.com/store/apps/details?id=...,Nice n fast,1,nice n fast,0.4215,0.0,0.417,0.583
3,https://play.google.com/store/apps/details?id=...,Won't update,1,won t update,0.5719,0.0,0.351,0.649
4,https://play.google.com/store/apps/details?id=...,Won't update,1,won t update,0.5719,0.0,0.351,0.649
5,https://play.google.com/store/apps/details?id=...,Won't update,1,won t update,0.5719,0.0,0.351,0.649
6,https://play.google.com/store/apps/details?id=...,Best app,1,best app,0.6369,0.0,0.192,0.808
7,https://play.google.com/store/apps/details?id=...,Is fast secure I love it,1,is fast secure i love it,0.7650,0.0,0.377,0.623
8,https://play.google.com/store/apps/details?id=...,Won't update. Ever.,1,won t update ever,0.5719,0.0,0.448,0.552
9,https://play.google.com/store/apps/details?id=...,Best app ever,1,best app ever,0.6369,0.0,0.323,0.677


### Using Textblob for Sentiment Analysis

In [16]:
from textblob import TextBlob

In [17]:
tb_df = neg_df.iloc[:,:4]
tb_df.drop(['Text'],axis = 1, inplace = True)
tb_df

,Review URL,Star,Cleaned_Text
0,https://play.google.com/store/apps/details?id=...,1,not able to update neither able to uninstall
1,https://play.google.com/store/apps/details?id=...,1,many unwanted ads
2,https://play.google.com/store/apps/details?id=...,1,very bad app
3,https://play.google.com/store/apps/details?id=...,1,many times i tried to update its not updating ...
4,https://play.google.com/store/apps/details?id=...,1,app is not getting update and it is not gettin...
...,...,...,...
1889,https://play.google.com/store/apps/details?id=...,1,it will not let me update samsung galaxy a20
1890,https://play.google.com/store/apps/details?id=...,1,every time updates screw things up i mean thi...
1891,https://play.google.com/store/apps/details?id=...,1,for every search google translator pops up ev...
1892,https://play.google.com/store/apps/details?id=...,1,bad version


In [18]:
polar = []
subject = []

def textblobSIA(data_column):

    for i in range(len(data_column)):
        obj = TextBlob(data_column[i])
        sentiment = obj.sentiment
        polar.append(sentiment[0])
        subject.append(sentiment[1])


textblobSIA(tb_df['Cleaned_Text'])
tb_df['Text_Polarity'] = polar
tb_df['Text_subjectivity'] = subject

In [19]:
tb_df = tb_df.loc[tb_df['Text_Polarity'] > 0.5]
tb_df

,Review URL,Star,Cleaned_Text,Text_Polarity,Text_subjectivity
30,https://play.google.com/store/apps/details?id=...,1,good,0.7,0.60
45,https://play.google.com/store/apps/details?id=...,1,good,0.7,0.60
64,https://play.google.com/store/apps/details?id=...,1,it is the best app for browsing,1.0,0.30
76,https://play.google.com/store/apps/details?id=...,1,best,1.0,0.30
77,https://play.google.com/store/apps/details?id=...,1,good,0.7,0.60
...,...,...,...,...,...
1806,https://play.google.com/store/apps/details?id=...,1,nice,0.6,1.00
1855,https://play.google.com/store/apps/details?id=...,1,good,0.7,0.60
1856,https://play.google.com/store/apps/details?id=...,1,good app,0.7,0.60
1859,https://play.google.com/store/apps/details?id=...,1,its good,0.7,0.60
